In [2]:
import cv2
import numpy as np
import re
from tkinter import *
from tkinter import messagebox
from PIL import Image, ImageTk


In [3]:
def is_valid_email(email):
    pattern = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
    return re.match(pattern, email)

def apply_transformation(image, transformation_type):
    rows, cols, _ = image.shape

    if transformation_type == "translate":
        M = np.float32([[1, 0, 100], [0, 1, 50]])
        transformed_image = cv2.warpAffine(image, M, (cols, rows))
        return transformed_image, "Translated_Image.jpg"
    
    elif transformation_type == "reflect":
        M = np.float32([[1, 0, 0], [0, -1, rows], [0, 0, 1]])
        reflected_img = cv2.warpPerspective(image, M, (int(cols), int(rows)))
        return reflected_img, "Reflected_Image.jpg"
    
    elif transformation_type == "rotate":
        rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), 30, 0.6)
        rotated_img = cv2.warpAffine(image, rotation_matrix, (cols, rows))
        return rotated_img, "Rotated_Image.jpg"
    
    elif transformation_type == "crop":
        cropped_img = image[50:200, 100:300]
        return cropped_img, "Cropped_Image.jpg"
    
    elif transformation_type == "gaussian_blur":
        blurred_img = cv2.GaussianBlur(image, (7, 7), 0)
        return blurred_img, "GaussianBlur_Image.jpg"
    
    else:
        return image, "Original_Image.jpg"

def show_image(img):
    # converting the image for tkinter
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    img_tk = ImageTk.PhotoImage(img_pil)

    # creating a new window to display the image
    image_window = Toplevel(root)
    image_window.title("Transformed Image")
    panel = Label(image_window, image=img_tk)
    panel.image = img_tk
    panel.pack()

def on_transform(): #function to handel the image transformations
    email = email_entry.get()
    if not is_valid_email(email):
        messagebox.showerror("Invalid Email", "Please enter a valid email address.")
        return

    try:
        age = int(age_entry.get())
        if age < 18:
            messagebox.showerror("Age Restriction", "Sorry! You must be at least 18 years old to access the collection.")
            return
    except ValueError:
        messagebox.showerror("Invalid Age", "Please enter a valid age.")
        return

    category = category_var.get().lower()
    if category not in ["traditional", "modern", "contemporary"]:
        messagebox.showerror("Invalid Category", "Please choose a valid art collection category.")
        return
    
    # Define image path based on category
    if category == "traditional":
        image_path = 'img/traditional_art.jpeg'
    elif category == "modern":
        image_path = 'img/modern_artjpeg.jpeg'
    elif category == "contemporary":
        image_path = 'img/contemporary_artjpeg.jpeg'
    
    try:
        original_image = cv2.imread(image_path)
        if original_image is None:
            messagebox.showerror("Image Error", f"Could not load image from {image_path}")
            return
        
        transformation_choice = transformation_var.get().lower()
        transformed_image, output_filename = apply_transformation(original_image.copy(), transformation_choice)

        # displaying the transformed image
        show_image(transformed_image)

        # saveing the transformed image
        cv2.imwrite(output_filename, transformed_image)
        messagebox.showinfo("Success", f"Transformed image saved as {output_filename}")
    
    except FileNotFoundError:
        messagebox.showerror("File Error", f"Image file not found at {image_path}")

# creating the main window
root = Tk()
root.title("Museum Art Collection App")

# making labels and entry widgets
Label(root, text="Enter your email:").grid(row=0, column=0)
email_entry = Entry(root, width=40)
email_entry.grid(row=0, column=1)

Label(root, text="Enter your age:").grid(row=1, column=0)
age_entry = Entry(root, width=40)
age_entry.grid(row=1, column=1)

Label(root, text="Choose an art collection category (Traditional, Modern, Contemporary):").grid(row=2, column=0, columnspan=2)
category_var = StringVar()
category_menu = OptionMenu(root, category_var, "Traditional", "Modern", "Contemporary")
category_menu.grid(row=3, column=0, columnspan=2)

Label(root, text="Choose a transformation:").grid(row=4, column=0)
transformation_var = StringVar()
transformation_menu = OptionMenu(root, transformation_var, "Translate", "Reflect", "Rotate", "Crop", "Gaussian Blur", "Original")
transformation_menu.grid(row=4, column=1)

# submit button
submit_button = Button(root, text="Apply Transformation", command=on_transform)
submit_button.grid(row=5, column=0, columnspan=2)

# starting the main GUI loop
root.mainloop()
